In [ ]:
import pandas as pd
import json

In [ ]:
import sys
sys.path.insert(0,"../../python/")
from rg17 import evaluate_toplist as et

# Load Player Accounts

In [ ]:
with open("/mnt/idms/fberes/network/online_ranker/roland_garros_updated_schedule/filtered_true_matches.json") as f:
    player_account_map = json.load(f)

## Select main account for players

# Load Schedule

In [ ]:
schedule_df = pd.read_csv("/mnt/idms/fberes/network/roland_garros/data/schedule_new_df.csv", sep="|")

In [ ]:
excluded_categories = ["boy", "girl", "wheelchair", "legends over 45"]

# Filter Schedule

In [ ]:
def filter_categories(match_cat, excluded_cats=excluded_categories):
    match_cat_lower = match_cat.lower()
    keep_this = True
    for cat in excluded_cats:
        if cat in match_cat_lower:
            keep_this = False
            break
    if not ("final" in match_cat_lower and "single" in match_cat_lower):
        keep_this = False
    return keep_this

In [ ]:
finals_df = schedule_df[schedule_df["matchHeader"].apply(filter_categories)]

In [ ]:
len(schedule_df), len(finals_df)

## Single finals

In [ ]:
finals_df

# Player name parts

In [ ]:
players = list(set(finals_df["playerName active"]).union(finals_df["playerName opponent"]))

In [ ]:
len(players)

In [ ]:
player_info_map = {}
for player in players:
    player_info_map[player] = {
        "name_parts": [p.lower() for p in player.split()],
        "accounts": ["@" + et.transform_account_name(a, remove_digits=False) for a in player_account_map[player]]
    }

## Show multi-account players

In [ ]:
for player, info in player_info_map.items():
    if len(info["accounts"]) > 1:
        print(player, info["accounts"])

In [ ]:
player_info_map

# Relevant player words

In [ ]:
finals_df.head(2)

In [ ]:
relevant_infos = []
for idx, row in finals_df.iterrows():
    date, time, winner, loser = row["date"], row["startDate"], row["playerName active"], row["playerName opponent"]
    for p1, p2, is_winner in [(winner, loser, 1),(loser, winner, 0)]:
        n1, n2 = player_info_map[p1]["name_parts"], player_info_map[p2]["name_parts"]
        acc1, acc2 = player_info_map[p1]["accounts"], player_info_map[p2]["accounts"]
        for w in n1+acc1:
            relevant_infos.append([date, time, p1, is_winner, w, p2, n2+acc2])
relevant_df = pd.DataFrame(relevant_infos, columns=["date","time","player","is_winner","key_word","opponent","relevant_words"])

In [ ]:
relevant_df

# TODO: player evaluation

   * Load occurences file
   * check whether relevant words are present or not..
   * DO a player evaluation for the words... (with NDCG)
   
# TODO: winner evaluation

   * for words like (win, won, triumph, victory etc.)
   * see whether winner name parts are present or not!!!